In [ ]:
!pip install skforecast
!pip install dask[dataframe]
!pip install lightgbm
!pip install matplotlib
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib
import joblib
import matplotlib.pyplot as plt
import warnings
import importlib


plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

warnings.filterwarnings('once')

In [ ]:
old_data = pd.read_csv('Fitabase Data 4.12.16-5.12.16/test_train_data.csv')
data = pd.read_csv('FitBit API Database/test_train_data_api_merged.csv')

print("Archivo : test_train_data.csv")
print(old_data)
print("\n")
print("Archivo : test_train_data_api_merged.csv")
print(data)

In [ ]:
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
data = data.set_index('Time')
data = data.asfreq('60S')
data = data.sort_index()
data

In [ ]:
steps = 5
lags =  5 * steps
lags

In [ ]:

datos_train = data[:-steps]
datos_test  = data[-steps:]

print(
    f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})"
)
print(
    f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})"
)

print(f'Número de filas con missing values : (datos_train) : {datos_train.isnull().any(axis=1).mean()}')
print(f'Número de filas con missing values:  (datos_test) : {datos_test.isnull().any(axis=1).mean()}')

In [ ]:
datos_train = datos_train.bfill();
print(datos_train)

In [ ]:
datos_test= datos_test.bfill();
print(datos_test)

In [ ]:
print(f'Número de filas con missing values : (datos_train) : {datos_train.isnull().any(axis=1).mean()}')
print(f'Número de filas con missing values:  (datos_test) : {datos_test.isnull().any(axis=1).mean()}')

# LightGBM

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = lgb.LGBMRegressor(objective='regression', random_state=123),
                lags      = lags
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])
predicciones

In [ ]:
datos_test['HeartRate']

In [ ]:
# Error test
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test['HeartRate'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
predicciones.plot(ax=ax, label='Predicciones')

ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('HeartRate')

plt.show()

# Tunning : LightGBM

In [ ]:
# lags_grid = [1, 20]

# param_grid = {'n_estimators': [100,200, 500],
#               'max_depth': [3, 5, 10]}

# resultados_grid = grid_search_forecaster(
#                         forecaster         = forecaster,
#                         y                  = datos_train['HeartRate'],
#                         param_grid         = param_grid,
#                         lags_grid          = lags_grid,
#                         steps              = steps,
#                         refit              = False,
#                         metric             = 'mean_squared_error',
#                         initial_train_size = int(len(datos_train)*0.5),
#                         fixed_train_size   = False,
#                         return_best        = True,
#                         n_jobs             = 'auto',
#                         verbose            = False
#                   )

# resultados_grid.head()

In [ ]:
# resultados_grid

In [ ]:
# best_params = {'n_estimators': resultados_grid.iloc[0]['n_estimators'],
#               'max_depth': resultados_grid.iloc[0]['max_depth']}

# tunned_Light_model = lgb.LGBMRegressor(objective='regression', random_state=123, **best_params)

In [ ]:
# forecaster = ForecasterAutoreg(
#                 regressor = tunned_Light_model,
#                 lags      = len(resultados_grid.iloc[0]['lags']),
                
#              )

# forecaster.fit(y=datos_train['HeartRate'])
# predicciones = forecaster.predict(steps=steps)

In [ ]:
# predicciones

In [ ]:
# error_mse = mean_squared_error(
#                 y_true = datos_test['HeartRate'],
#                 y_pred = predicciones
#             )

# print(f"Error de test (mse) {error_mse}")

In [ ]:
# fig, ax = plt.subplots(figsize=(6, 2.5))

# datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
# datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
# predicciones.plot(ax=ax, label='Predicciones')

# ax.legend()
# ax.set_xlabel('Time')
# ax.set_ylabel('HeartRate')

# plt.show()

# XGBoost

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(random_state=123),
                lags      = lags
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])

In [ ]:
predicciones

In [ ]:
datos_test

In [ ]:
# Error test
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test['HeartRate'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
predicciones.plot(ax=ax, label='Predicciones')

ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('HeartRate')

plt.show()

# Tunning XGBoost

In [ ]:
# lags_grid = [1, 20]

# param_grid = {'n_estimators': [100,200, 500],
#               'max_depth': [3, 5, 10]}

# resultados_grid = grid_search_forecaster(
#                         forecaster         = forecaster,
#                         y                  = datos_train['HeartRate'],
#                         param_grid         = param_grid,
#                         lags_grid          = lags_grid,
#                         steps              = steps,
#                         refit              = False,
#                         metric             = 'mean_squared_error',
#                         initial_train_size = int(len(datos_train)*0.5),
#                         fixed_train_size   = False,
#                         return_best        = True,
#                         n_jobs             = 'auto',
#                         verbose            = False
#                   )

# resultados_grid.head()

In [ ]:
# best_params = {'n_estimators': resultados_grid.iloc[0]['n_estimators'],
#               'max_depth': resultados_grid.iloc[0]['max_depth']}

# tunned_XGBoost_model = XGBRegressor(random_state=123, **best_params)

In [ ]:
# forecaster = ForecasterAutoreg(
#                 regressor = tunned_XGBoost_model,
#                 lags      = len(resultados_grid.iloc[0]['lags']),
                
#              )

# forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])

In [ ]:
# predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])

In [ ]:
# predicciones

In [ ]:
# datos_test

In [ ]:
# # Error test
# # ==============================================================================
# error_mse = mean_squared_error(
#                 y_true = datos_test['HeartRate'],
#                 y_pred = predicciones
#             )

# print(f"Error de test (mse): {error_mse}")

In [ ]:
# fig, ax = plt.subplots(figsize=(6, 2.5))

# datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
# datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
# predicciones.plot(ax=ax, label='Predicciones')

# ax.legend()
# ax.set_xlabel('Time')
# ax.set_ylabel('HeartRate')

# plt.show()

# KNN Regressor

In [ ]:
n_neighbors = 5
knn_model = KNeighborsRegressor(n_neighbors=n_neighbors)

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = knn_model,
                lags      = lags
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])

In [ ]:
predicciones

In [ ]:
datos_test

In [ ]:
# Error test
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test['HeartRate'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
predicciones.plot(ax=ax, label='Predicciones')

ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('HeartRate')

plt.show()

# Random Forest Regressor

In [ ]:
n_estimators = 100 
max_depth = None 

random_forest_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=123)

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = random_forest_model,
                lags      = lags
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])

In [ ]:
predicciones

In [ ]:
datos_test

In [ ]:
# Error test
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test['HeartRate'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
predicciones.plot(ax=ax, label='Predicciones')

ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('HeartRate')

plt.show()

# MLP Regressor

In [ ]:
MLP_model = MLPRegressor(random_state=123, max_iter=2000)

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = MLP_model,
                lags      = lags
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])

In [ ]:
predicciones

In [ ]:
datos_test

In [ ]:
# Error test
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test['HeartRate'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
predicciones.plot(ax=ax, label='Predicciones')

ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('HeartRate')

plt.show()

In [ ]:
# # Backtesting
# # ==============================================================================
# n_backtesting = steps*lags 

# metrica, predicciones_backtest = backtesting_forecaster(
#                                     forecaster         = forecaster,
#                                     y                  = datos_train['HeartRate'],
#                                     initial_train_size = len(datos_train) - n_backtesting,
#                                     fixed_train_size   = False,
#                                     steps              = steps,
#                                     refit              = True,
#                                     metric             = 'mean_squared_error',
#                                     verbose            = True
#                                  )

# print(f"Error de backtest: {metrica}")

In [ ]:
# # Gráfico de predicciones de backtest vs valores reales
# # ==============================================================================
# fig, ax = plt.subplots(figsize=(6, 2.5))
# datos_train.loc[predicciones_backtest.index, 'HeartRate'].plot(ax=ax, label='test')
# predicciones_backtest.plot(ax=ax, label='predicciones')
# ax.legend();

# SVM: Support Vector Machine

In [ ]:
model_SVMR = SVR(kernel='rbf', gamma='auto', epsilon=0.1)

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = model_SVMR,
                lags      = lags
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps,  exog=datos_test[['Calories', 'Steps', 'Distance']])
predicciones

In [ ]:
datos_test

In [ ]:
# Error test
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test['HeartRate'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))

datos_train['HeartRate'].tail(7).plot(ax = ax, label = 'Datos train')
datos_test['HeartRate'].plot(ax=ax, label='Datos_test')
predicciones.plot(ax=ax, label='Predicciones')

ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('HeartRate')

plt.show()